In [2]:
import pandas as pd
import numpy as np
import requests
from lxml import etree

#########################获取网址中的行政区划代码###################################################
r= requests.get('http://www.mca.gov.cn/article/sj/xzqh/2020/20201201.html')
html = etree.HTML(r.text)
regions = []
for tr in html.xpath('//tr'):
    code=tr.xpath('td[2]/text()')
    if code:
        name=tr.xpath('td[3]/text()')[0]
        regions.append([code[0], name])
#########################生成查询字典和列表###################################################
# 查询字典用来级联查询地址
d = {}
province = None
province_str = ''
city = None
city_str = ''
# 是否直辖市
municipality = False
# 列表用来存放到数据库，以显示给定行政区划代码的地址
li = []
for code, region in regions[1:]:
    # 判断级别是省、市、县
    if code.endswith('0000'):
        d[region] = {}
        province = d[region]
        province_str = region
        li.append([code, province_str])
        municipality = False
        last_level = 1
    elif code.endswith('00'):
        province[region] = {}
        city = province[region]
        city_str = region
        li.append([code, province_str, city_str])
        last_level = 2
    else:
        if municipality is True:
            province[region] = {}
            li.append([code, province_str, region])
        else:
            if last_level == 1:
                municipality = True
                province[region] = {}
                li.append([code, province_str, region])
            elif last_level == 2:
                city[region] = {}
                li.append([code, province_str, city_str, region])
            elif last_level == 3:
                city[region] = {}
                li.append([code, province_str, city_str, region])
        last_level = 3
#########################根据手动输入的礼县各社区的小区，给字典和列表添加礼县的详细信息#######################
lx = pd.read_excel('./lixian.xlsx', converters={'code':str})
lx = np.array(lx).tolist()
new = li +lx
df = pd.DataFrame(new)
df = pd.DataFrame(df)
df.where(df.notnull(), None) 
df=df.sort_values(by=0)
dic = d['甘肃省']['陇南市']['礼县']
country = None
village = None
neighborhood = None
for row in lx:
    country = row[4]
    village = row[5]
    neighborhood = row[6]
    if isinstance(country, str):
        if country not in dic:
            dic[country] = {}
    else:
        continue
    if isinstance(village, str):
        if village not in dic[country]:
            dic[country][village] = {}
    else:
        continue
    if isinstance(neighborhood, str):
        if neighborhood not in dic[country][village]:
            dic[country][village][neighborhood] = {}
    else:
        continue
#########################持久化保存字典#######################
import json
with open('region_code_dict.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(d, ensure_ascii=False))
#########################持久化保存列表#######################
df.to_excel('region_code_list.xlsx', index=False, header=['code', 'province', 'city', 'county', 'country', 'village', 'neighborhood'])
